In [ ]:
'''    
    Analyses of ribosome profiling data for mapping KS SecM stalling reporter
    
    Copyright (C) 2021  Allen Buskirk, buskirk@jhmi.edu
    
    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.
    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.
    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <http://www.gnu.org/licenses/>.
'''
## this notebook is for the analysis of plasmid reads in profiling libraries
## harboring the NanoLuc-shortSecM/IRAGP reporter

%load_ext autoreload
%autoreload 2
import os
import ribo_utilities
from BCBio import GFF

FP_names = [
    'ac21', 'ac22', 'ac23',
    'ac24', 'ac25', 'ac26'
    ]

inpath      = '/home/annie/profiling_data/hrpA_IRAGPK_rep1_data/'    
path_script = '/home/annie/code/profiling/code_hrpA/'

paths_in = {
    'path_fastq'     : '/home/annie/profiling_data/hrpA_lowERY_rep1_data/fastq/', 
    'path_gff'       : path_script + 'coli2.gff',
    'path_bowtie'    : 'bowtie',
    'index_ladder'   : path_script + 'indexes/ladder/ladder',
    'index_trna'     : path_script + 'indexes/MG1655/MG1655_tRNA',
    'index_rrna'     : path_script + 'indexes/MG1655/MG1655_rRNA',
    'index_plasmid'  : path_script + 'indexes/IRAGP/IRAGP', # plasmid index for alignment
    'path_fasta'     : path_script + 'indexes/IRAGP/IRAGP.fasta', # plasmid fasta for density
    'path_totreads'  : '/home/annie/profiling_data/hrpA_lowERY_rep1_data/density/' # reads.csv from whole genome density, which is needed for normalizing plasmid density
    }

paths_out = {
    'path_filter'       : '/home/annie/profiling_data/hrpA_lowERY_rep1_data/filterdata/',
    'path_ladder'       : inpath  + 'alignments/ladder/',
    'path_trna'         : inpath  + 'alignments/tRNA/',
    'path_rrna'         : inpath  + 'alignments/rRNA/',
    'path_chr'          : inpath  + 'alignments/chr/',
    'path_plasmid'      : inpath  + 'alignments/IRAGP/', # path for plasmid alignment output
    'path_temp'         : inpath  + 'tmpds/',
    'path_density'      : inpath  + 'density/',
    'path_wig'          : inpath  + 'wigfiles/',
    'path_log'          : inpath  + 'log/',
    'path_genelists'    : inpath  + 'analysis/genelists/',
    'path_pauses'       : inpath  + 'analysis/pauses/'
    }

for i in paths_out:
    if not os.path.exists(paths_out[i]):
        os.makedirs(paths_out[i])



In [ ]:
## use cutadapt 2.1 version to remove the linker from the reads in the FASTQ files 
## and the UMI 4 nt before and 6 nt after the footprint itself


import os

minlength = 10

linker = 'NNNNNNCACTCGGGCACCAAGGAC'

files = FP_names

#print "log saved at paths_out['path_filter']"
# teach cutadapt to save output or save by hand from the notebook terminal

for fname in files:
    file_in = paths_in['path_fastq'] + fname + '.fastq'
    file_out = paths_out['path_filter'] + fname + '_filtered_trimmed.fastq'

    cutadapt_command = 'cutadapt -a %s --discard-untrimmed %s | cutadapt -u 4 -m %d - > %s' % (
        linker, 
        file_in, 
        minlength,
        file_out
        )

    os.system(cutadapt_command)
    print fname + " completed"


In [ ]:
# alignments using bowtie 1.2.2
# this function aligns to a ladder index, then the tRNA index, then the rRNA index, then the chromosome
# tRNA and rRNA match alignments stored in the tmpds folder
# makes tmpds folder and alignment folders automatically

import os
import sys

files = FP_names

######################################################

path_l = paths_out['path_ladder']
path_t = paths_out['path_trna']
path_r = paths_out['path_rrna']
path_plasmid = paths_out['path_plasmid']
path_temp = paths_out['path_temp']

if os.path.exists(paths_out['path_filter']+files[0]+'_filtered_trimmed.fastq'):
    trimmed = '_filtered_trimmed.fastq '
elif os.path.exists(paths_out['path_filter']+files[0]+'-trimmed.fastq'):
    trimmed = '-trimmed.fastq '
else:
    sys.exit("error: trimmed file name")
    
for fname in files:
    print fname
    
    #first, align to ladder index to subtract any ladder oligos that were cloned
    bowtie_l = paths_in['path_bowtie'] + ' -v 2 -y -m 1 -a --best --strata -S -p 2 --un '
    bowtie_l += path_l + fname + '_nomatch.fastq --max ' + path_l + fname + '_multi.fastq --al '
    bowtie_l += path_l + fname + '_match.fastq ' + paths_in['index_ladder'] + ' '
    bowtie_l += paths_out['path_filter'] + fname + trimmed + ' 1> ' + path_temp + fname + '_ladder_match.SAM'
    bowtie_l += ' 2> ' + paths_out['path_log'] + fname
    os.system(bowtie_l)
    os.system("echo 'End ladder alignment' >> " + paths_out['path_log'] + fname)

    # second, align to tRNA index
    bowtie_tRNA = paths_in['path_bowtie'] + ' -v 2 -y -m 1 -a --best --strata -S -p 2 --un '
    bowtie_tRNA += path_t + fname + '_nomatch.fastq --max ' + path_t + fname + '_multi.fastq --al '
    bowtie_tRNA += path_t + fname + '_match.fastq ' + paths_in['index_trna'] + ' '
    bowtie_tRNA += path_l + fname + '_nomatch.fastq ' + ' 1> ' + path_temp + fname + '_tRNA_match.SAM'
    bowtie_tRNA += ' 2>> ' + paths_out['path_log'] + fname 
    os.system(bowtie_tRNA)
    os.system("echo 'End tRNA alignment' >> " + paths_out['path_log'] + fname)

    # third, align to the rRNA index
    bowtie_rRNA = paths_in['path_bowtie'] + ' -v 2 -y -m 1 -a --best --strata -S -p 2 --un '
    bowtie_rRNA += path_r + fname + '_nomatch.fastq --max ' + path_r + fname + '_multi.fastq --al '
    bowtie_rRNA += path_r + fname + '_match.fastq ' + paths_in['index_rrna'] + ' '
    bowtie_rRNA += path_t + fname + '_nomatch.fastq ' + ' 1> ' + path_temp + fname + '_rRNA_match.SAM'
    bowtie_rRNA += ' 2>> ' + paths_out['path_log'] + fname 
    os.system(bowtie_rRNA)
    os.system("echo 'End rRNA alignment' >> " + paths_out['path_log'] + fname)

    # finally align to the plasmid reporter index
    bowtie_plasmid = paths_in['path_bowtie'] + ' -v 2 -y -m 1 -a --best --strata -S -p 2 --un '
    bowtie_plasmid += path_plasmid + fname + '_nomatch.fastq --max ' + path_plasmid + fname + '_multi.fastq --al '
    bowtie_plasmid += path_plasmid + fname + '_match.fastq ' + paths_in['index_plasmid'] + ' '
    bowtie_plasmid += path_r + fname + '_nomatch.fastq ' + ' 1> ' + path_plasmid + fname + '_match.SAM'
    bowtie_plasmid += ' 2>> ' + paths_out['path_log'] + fname 
    os.system(bowtie_plasmid)
    os.system("echo 'End plasmid alignment' >> " + paths_out['path_log'] + fname)

In [ ]:
### creating ribosome density files for PLASMID, saving pickle file and WIG file

files = FP_names
minlength = 10
maxlength = 35
threads = 4

ribo_utilities.run_density_plasmid(files, minlength, maxlength, threads, paths_in, paths_out)